In [29]:
import os

import pandas as pd
import re

import random

import plotly
import plotly.offline
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [8]:
results_dir= 'D:/Complete_points/'

In [10]:
files= os.listdir(results_dir)


In [25]:
axons=[os.path.join(results_dir,i) for i in files if '_axons.csv' in i]


    x   y    z  atlas_ID
0  80  62  244       981
1  80  62  244       981
2  80  61  245       981
3  80  61  245       981
4  80  61  245       981
    x   y    z  atlas_ID
0  90  56  237       981
1  90  56  237       981
2  90  56  237       981
3  90  56  237       981
4  89  57  237       981


In [35]:
random.randint(1,len(axons))


1

In [38]:
fig=go.Figure()

for count,names in enumerate(axons):
    data_pd=pd.read_csv(names)
    name= re.search('\D{2}[0-9]{3}', names)[0]
    fig.add_trace(go.Scatter3d( 
        x=data_pd['x'],
        y=data_pd['y'],
        z=data_pd['z'],
        mode='markers',
        name=name,
        marker=dict(
            size=1,
            color=count,
            opacity=0.5
        ),
        hovertext= data_pd['atlas_ID'],
        hoverinfo='text'
    ))

camera = dict(
        eye=dict(x=0, y=-1, z=0)
    )

    #Need to change the axis title depending on the direction and orientation of the origional image (ie, imaging started from cerebellum vs. imaging started from olfactorybulb)
fig.update_layout(scene_camera=camera, 
                      scene=dict(
                        xaxis_title='Smaller=more medial',
                        yaxis_title='Smaller=more dorsal',
                        zaxis_title='Smaller=more anterior'))
plot(fig)

'temp-plot.html'

In [ ]:
def make_scatter(axons, dendrites):
        
    fig=go.Figure()
    fig.add_trace(go.Scatter3d( 
        x=axons[::3]['x'],
        y=axons[::3]['y'],
        z=axons[::3]['z'],
        mode='markers',
        name='Axons',
        marker=dict(
            size=1,
            color='blue',
            opacity=0.5
        ),
        hovertext= axons[::3]['name'],
        hoverinfo='text'
    ))
    
    fig.add_trace(go.Scatter3d(
        x=dendrites['x'],
        y=dendrites['y'],
        z=dendrites['z'],
        mode='markers',
        name='Dendrites',
        marker=dict(
            size=2,
            color='red',
            opacity=0.5),
        hovertext= dendrites['name'],
        hoverinfo='text'
    ))

#Here we want horizontal view with dorsal surface toward us
#first we set a xz view for coronal section this is done by setting x and z as 0 in eye
# then, for y we want the smaller values to point toward us (because in our original coronal images dorsal surface is pointed up)
    camera = dict(
        eye=dict(x=0, y=-1, z=0)
    )


    #Need to change the axis title depending on the direction and orientation of the origional image (ie, imaging started from cerebellum vs. imaging started from olfactorybulb)
    fig.update_layout(scene_camera=camera, 
                      scene=dict(
                        xaxis_title='Smaller=more medial',
                        yaxis_title='Smaller=more dorsal',
                        zaxis_title='Smaller=more anterior'))
    plot(fig,filename=outdir+'/'+'3Dplot.html')
    